# Linear programming formulation of two-sided matching problems

In this notebook, I solve the stable matching problem with the linear programming formulation with Julia/JuMP.
The formulation is based on [Online and Matching-Based Market Design 1.5](https://www.cambridge.org/core/books/online-and-matchingbased-market-design/604CA9FF1396C489D6497CF336368524).

## Setting

I consider an instance with the same number of workers ($w$) and firms ($f$), and I consider one-to-one complete matchings.
Let the sets of workers and firms be $W$ and $F$.
Assume that the preferences are strict.
A sufficient condition so that a worker-firm pair does not form a blocking pair in this matching is that, if $w$ is matched to firm $f'$ such that another firm $f$ is preferred to $f'$ by $w$, then $f$ should be matched to another worker $w'$ who is preferred to $w$ by $f$.

Using this, the linear programming for finding a stable matching is formulated as follows:

\begin{align*}
    &\max_{x_{wf}} \quad 0 \\
    \text{subject to} &\sum_f x_{wf} = 1 \quad \forall w \in W \\
                      &\sum_w x_{wf} = 1 \quad \forall f \in F \\
                      &\sum_{\text{$f'$; $f$ preferred to $f'$ by $w$}} x_{wf'} - \sum_{w'; \text{$w'$ preferred to $w$ by $f$}} x_{w'f} \le 0 \quad \forall w \in W, \forall f \in F \\
                      &x_{wf} \ge 0 \quad \forall w \in W, \forall f \in F.
\end{align*}

The first two constraints are for the perfect matching, and the third constraint is for the stable matching.

## Julia implementation

In [5]:
using BenchmarkTools
using Random
using JuMP
using HiGHS
using DeferredAcceptance

First I consider an instance with 5 workers and 5 firms.

In [23]:
Random.seed!(123)
N = 5;

workers_pref =  mapslices(x -> sortperm(x), rand(Float64, (N, N)), dims=1);
firms_pref =  mapslices(x -> sortperm(x), rand(Float64, (N, N)), dims=1);

The workers' preferences are as follows (column: worker, row: firm, and $1$ means the firm is the best choice for the worker):

In [24]:
workers_pref

5×5 Matrix{Int64}:
 5  3  1  5  5
 4  5  5  2  3
 3  4  2  3  2
 1  2  4  1  1
 2  1  3  4  4

And the firms' preferences are as follows (column: firm, row: worker, and $1$ means the worker is the best choice for the firm):

In [25]:
firms_pref

5×5 Matrix{Int64}:
 4  3  1  5  2
 2  1  5  2  1
 3  5  4  3  5
 1  2  2  1  3
 5  4  3  4  4

## JuMP formulation for LP

In [40]:
model = Model(HiGHS.Optimizer);

@variable(model, x[1:N, 1:N] >= 0);

@objective(model, Max, 0)

# constraints for perfect matching
@constraint(model, [i = 1:N], sum(x[i, j] for j in 1:N) == 1);
@constraint(model, [j = 1:N], sum(x[i, j] for i in 1:N) == 1);

# constraint for stable matching
@constraint(model, [i=1:N, j=1:N], 
    sum(x[i, k] * (workers_pref[j, i] < workers_pref[k, i]) for k in 1:N) 
    - sum(x[l, j] * (firms_pref[i, j] > firms_pref[l, j]) for l in 1:N) <= 0);

In [41]:
@time optimize!(model)

Presolving model
16 rows, 13 cols, 57 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-35); columns 0(-25); elements 0(-150) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
  0.002227 seconds (518 allocations: 49.797 KiB)


In [42]:
value.(x)

5×5 Matrix{Float64}:
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 0.0  1.0  0.0  0.0  0.0

This means that the pairs are ($w_1$, $f_3$), ($w_2$, $f_5$), ($w_3$, $f_1$), ($w_4$, $f_4$), and ($w_5$, $f_2$).
Using the `isstable` function in [`DeferredAcceptance` package](https://juliapackages.com/p/deferredacceptance), I confirm that this matching is actually stable:

In [43]:
isstable(workers_pref, firms_pref, ones(Int, N), mapslices(y -> findfirst(y .== 1), value.(x), dims=2)[:])

true